In [0]:
!pip install tensorflow-hub
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 26kB/s 
     |████████████████████████████████| 450kB 62.1MB/s 
     |████████████████████████████████| 3.9MB 60.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4c63335cdc140fd3d23cebf4f4322db56daf0b9f27c9b9d1813ca8f085056811
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

In [0]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install bert-for-tf2
!pip install sentencepiece

In [0]:
import tensorflow as tf
# tf.enable_eager_execution()
# tf.executing_eagerly() 
# sess = tf.Session()
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
import bert
import tokenization
import tensorflow_hub as hub

##
#Libraries
import pandas as pd
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from spacy.lang.pt.examples import sentences
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.decomposition import PCA
import pickle
#nlp = spacy.load('pt')
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 
import matplotlib.pyplot as plt
from keras.layers import Flatten

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
print(tf.__version__)
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

2.1.0
TF version: 2.1.0
Hub version: 0.8.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [0]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/1",trainable=True)
vocab_file= bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocab_file, to_lower_case)

In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def overSamplingRISKs(df,rate):
  X = df 
  not_HIGH = df[df.RISK!="HIGH"]
  HIGH = df[df.RISK=="HIGH"]
  oversample_by= int((rate*((len(df[df.RISK!="HIGH"])+len(df[df.RISK=="HIGH"]))-(len(df[df.RISK=="HIGH"]))))/(1-rate))
  # OVERsample minority
  high_upsampled = resample(HIGH,
                            replace=True, # sample with replacement
                            n_samples=oversample_by, # match number in majority class
                            random_state=27) # reproducible results

  upsampled = pd.concat([not_HIGH, high_upsampled])
  # check new class counts
  print( upsampled.RISK.value_counts() )

  return upsampled

In [0]:
#Paths  #Import FILES
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'
filename= DATASETS_FOLDER+'2016_DF_cleaned_not_balanced.pkl'
#df.to_pickle(filename)
df=pd.read_pickle(filename)

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
datatrain,datatest= train_test_split(df, test_size=0.30, random_state=42)
datatrain = overSamplingRISKs(datatrain,0.30)

MEDIUM-LOW    12467
HIGH           5343
Name: RISK, dtype: int64


In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
train_input = bert_encode(datatrain['sent_token_final'].values, tokenizer, max_len=160)
test_input = bert_encode(datatest['sent_token_final'].values, tokenizer, max_len=160)

train_labels = datatrain['FinalLabel']
test_labels = datatest['FinalLabel']



In [0]:
test_labels.groupby(test_labels).count()


FinalLabel
0     194
1    5337
Name: FinalLabel, dtype: int64

In [0]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    clf_output = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
    clf_output = tf.keras.layers.Dropout(0.2)(clf_output)
    clf_output = tf.keras.layers.Flatten()(clf_output)
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
model = build_model(bert_layer, max_len=160)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 768), (None, 177853441   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
model.fit(train_input, np.array(train_labels),validation_data=(test_input,np.array(test_labels)),epochs=10,batch_size=32)

Train on 17810 samples, validate on 5531 samples
Epoch 1/10
17810/17810 [==============================] - 379s 21ms/sample - loss: 0.6158 - accuracy: 0.7000 - val_loss: 0.3346 - val_accuracy: 0.9649
Epoch 2/10
17810/17810 [==============================] - 378s 21ms/sample - loss: 0.6138 - accuracy: 0.7000 - val_loss: 0.4051 - val_accuracy: 0.9649
Epoch 3/10
 3072/17810 [====>.........................] - ETA: 4:44 - loss: 0.6092 - accuracy: 0.7035

In [0]:
predictions_TRAIN=model.predict(train_input, batch_size=32, verbose=1)
prediction= model.predict(test_input, batch_size=32, verbose=1)



5531/5531 [==============================] - 34s 6ms/sample


In [0]:
# %run '/content/drive/My Drive/Fraud detection in internal auditing/Colab Notebooks/NLP functions_Gus.ipynb'
%run '/content/drive/My Drive/Colab Notebooks/NLP-4-Classification-FUNCTIONS.ipynb'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
predicionsTRAIN_2= predictions_TRAIN.round().astype(int)
acc1, pre1, rec1, f1s1 =get_metrics(train_labels,predicionsTRAIN_2)

predicions_2= prediction.round().astype(int)
acc, pre, rec, f1s =get_metrics(test_labels,predicions_2)

Accuracy: 0.7  Precision: 0.7  Recall: 1.0  F1 Score: 0.82
Accuracy: 0.96  Precision: 0.96  Recall: 1.0  F1 Score: 0.98
